# Data Preparation

Notebook que irá conter o código utilizado para a preparação dos dados do projeto.

os dados "crus" estão localizados em **/src/data/raw-data** e os dados tratados devem ser salvos em **/src/data/processed-data** 

## Initial Setup

In [1]:
import pandas as pd
import numpy as np

import pickle as pkl

from joblib import Parallel, delayed

In [2]:
# File paths for working locally
raw_data_path = '../data/raw-data/'
processed_data_path = '../data/processed-data/'

In [3]:
# # Uncomment this cell if running on Google Colab
# from google.colab import drive
# drive.mount('/content/drive/')
# raw_data_path = '/content/drive/MyDrive/datasets/mlb-player-digital-engagement-forecasting/'

## Loading the auxiliary Datasets

In [4]:
dataset_names = {
    'Awards': 'awards.csv', 
    'Example': 'example_test.csv', 
    'Players': 'players.csv',
    'Seasons': 'seasons.csv', 
    'Teams': 'teams.csv', 
    'Train': 'train_updated.csv'
}
for key in dataset_names:
  dataset_names[key] = raw_data_path + dataset_names[key]
dataset_names

{'Awards': '../data/raw-data/awards.csv',
 'Example': '../data/raw-data/example_test.csv',
 'Players': '../data/raw-data/players.csv',
 'Seasons': '../data/raw-data/seasons.csv',
 'Teams': '../data/raw-data/teams.csv',
 'Train': '../data/raw-data/train_updated.csv'}

### Players

In [5]:
df_players = pd.read_csv(dataset_names['Players'])
df_players.head()

,playerId,playerName,DOB,mlbDebutDate,birthCity,birthStateProvince,birthCountry,heightInches,weight,primaryPositionCode,primaryPositionName,playerForTestSetAndFuturePreds
0,665482,Gilberto Celestino,1999-02-13,2021-06-02,Santo Domingo,NaN,Dominican Republic,72,170,8,Outfielder,False
1,593590,Webster Rivas,1990-08-08,2021-05-28,Nagua,NaN,Dominican Republic,73,219,3,First Base,True
2,661269,Vladimir Gutierrez,1995-09-18,2021-05-28,Havana,NaN,Cuba,73,190,1,Pitcher,True
3,669212,Eli Morgan,1996-05-13,2021-05-28,Rancho Palos Verdes,CA,USA,70,190,1,Pitcher,True
4,666201,Alek Manoah,1998-01-09,2021-05-27,Homestead,FL,USA,78,260,1,Pitcher,True


In [6]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2061 entries, 0 to 2060
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   playerId                        2061 non-null   int64 
 1   playerName                      2061 non-null   object
 2   DOB                             2061 non-null   object
 3   mlbDebutDate                    2025 non-null   object
 4   birthCity                       2061 non-null   object
 5   birthStateProvince              1516 non-null   object
 6   birthCountry                    2061 non-null   object
 7   heightInches                    2061 non-null   int64 
 8   weight                          2061 non-null   int64 
 9   primaryPositionCode             2061 non-null   object
 10  primaryPositionName             2061 non-null   object
 11  playerForTestSetAndFuturePreds  2057 non-null   object
dtypes: int64(3), object(9)
memory usage: 193.3+ KB


Analisando o dataset, foram removidas a coluna `playerName`, por ela já estar representada pela coluna `playerId` e a coluna `birthStateProvince` pela quantidade de valores nulos que a mesma possui.

In [7]:
# Select the columns that will be used on the df and renames them according to the pattern
cols = {
    'playerId': 'IdPlayer',
    'DOB': 'DtBirth',
    'mlbDebutDate': 'DtMlbDebut',
    'birthCity': 'NmCity',
    #'birthStateProvince': 'NmState',
    'birthCountry': 'NmCountry',
    'heightInches': 'NuHeight',
    'weight': 'NuWeight',
    'primaryPositionCode': 'CdPrimaryPosition',
    'primaryPositionName': 'NmPrimaryPosition',
    'playerForTestSetAndFuturePreds': 'FlgForTestAndPred'
}
df_players = df_players[list(cols)]
df_players.columns = list(cols.values())
df_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2061 entries, 0 to 2060
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   IdPlayer           2061 non-null   int64 
 1   DtBirth            2061 non-null   object
 2   DtMlbDebut         2025 non-null   object
 3   NmCity             2061 non-null   object
 4   NmCountry          2061 non-null   object
 5   NuHeight           2061 non-null   int64 
 6   NuWeight           2061 non-null   int64 
 7   CdPrimaryPosition  2061 non-null   object
 8   NmPrimaryPosition  2061 non-null   object
 9   FlgForTestAndPred  2057 non-null   object
dtypes: int64(3), object(7)
memory usage: 161.1+ KB


In [8]:
# pd.to_pickle(df_players, processed_data_path + 'players.pkl')
del df_players

### Teams

In [9]:
df_teams = pd.read_csv(dataset_names['Teams'])
df_teams.head()

,id,name,teamName,teamCode,shortName,abbreviation,locationName,leagueId,leagueName,divisionId,divisionName,venueId,venueName
0,108,Los Angeles Angels,Angels,ana,LA Angels,LAA,Anaheim,103,American League,200,American League West,1,Angel Stadium
1,109,Arizona Diamondbacks,D-backs,ari,Arizona,ARI,Phoenix,104,National League,203,National League West,15,Chase Field
2,110,Baltimore Orioles,Orioles,bal,Baltimore,BAL,Baltimore,103,American League,201,American League East,2,Oriole Park at Camden Yards
3,111,Boston Red Sox,Red Sox,bos,Boston,BOS,Boston,103,American League,201,American League East,3,Fenway Park
4,112,Chicago Cubs,Cubs,chn,Chi Cubs,CHC,Chicago,104,National League,205,National League Central,17,Wrigley Field


In [10]:
df_teams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            30 non-null     int64 
 1   name          30 non-null     object
 2   teamName      30 non-null     object
 3   teamCode      30 non-null     object
 4   shortName     30 non-null     object
 5   abbreviation  30 non-null     object
 6   locationName  30 non-null     object
 7   leagueId      30 non-null     int64 
 8   leagueName    30 non-null     object
 9   divisionId    30 non-null     int64 
 10  divisionName  30 non-null     object
 11  venueId       30 non-null     int64 
 12  venueName     30 non-null     object
dtypes: int64(4), object(9)
memory usage: 3.2+ KB


In [11]:
cols = {
     'id': 'IdTeam'
    ,'locationName': 'NmLocation'
    ,'leagueId': 'IdLeague'
    ,'divisionId': 'IdDivision'
    ,'venueId': 'IdVenue'
}
df_teams = df_teams[list(cols.keys())]
df_teams.columns = list(cols.values())
df_teams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   IdTeam      30 non-null     int64 
 1   NmLocation  30 non-null     object
 2   IdLeague    30 non-null     int64 
 3   IdDivision  30 non-null     int64 
 4   IdVenue     30 non-null     int64 
dtypes: int64(4), object(1)
memory usage: 1.3+ KB


In [12]:
# pd.to_pickle(df_teams, processed_data_path + 'teams.pkl')
del df_teams

### Awards

In [13]:
df_awards = pd.read_csv(dataset_names['Awards'])
df_awards.head()

,awardDate,awardSeason,awardId,awardName,playerId,playerName,awardPlayerTeamId
0,2017-12-21,2017,WARRENSPAHN,Warren Spahn Award,477132,Clayton Kershaw,119.0
1,2017-12-20,2017,MILBORGAS,MiLB.com Organization All-Star,474319,Brandon Snyder,120.0
2,2017-12-20,2017,MILBORGAS,MiLB.com Organization All-Star,592530,Jose Marmolejos,120.0
3,2017-12-20,2017,MILBORGAS,MiLB.com Organization All-Star,593833,Wander Suero,120.0
4,2017-12-20,2017,MILBORGAS,MiLB.com Organization All-Star,600466,Raudy Read,120.0


In [14]:
df_awards.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11256 entries, 0 to 11255
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   awardDate          11256 non-null  object 
 1   awardSeason        11256 non-null  int64  
 2   awardId            11256 non-null  object 
 3   awardName          11256 non-null  object 
 4   playerId           11256 non-null  int64  
 5   playerName         11256 non-null  object 
 6   awardPlayerTeamId  11243 non-null  float64
dtypes: float64(1), int64(2), object(4)
memory usage: 615.7+ KB


In [15]:
cols = {
    'awardId': 'IdAward',
    'awardDate': 'DtAward',
    'awardSeason': 'DtAwardSeason',
    'playerId': 'IdPlayer',
    'awardPlayerTeamId': 'IdTeam'
}
df_awards = df_awards[list(cols)]
df_awards.columns = list(cols.values())
df_awards.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11256 entries, 0 to 11255
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   IdAward        11256 non-null  object 
 1   DtAward        11256 non-null  object 
 2   DtAwardSeason  11256 non-null  int64  
 3   IdPlayer       11256 non-null  int64  
 4   IdTeam         11243 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 439.8+ KB


In [16]:
# pd.to_pickle(df_awards, processed_data_path + 'awards.pkl')
del df_awards

### Seasons

In [17]:
df_seasons = pd.read_csv(dataset_names['Seasons'])
df_seasons.head()

,seasonId,seasonStartDate,seasonEndDate,preSeasonStartDate,preSeasonEndDate,regularSeasonStartDate,regularSeasonEndDate,lastDate1stHalf,allStarDate,firstDate2ndHalf,postSeasonStartDate,postSeasonEndDate
0,2017,2017-04-02,2017-11-01,2017-02-22,2017-04-01,2017-04-02,2017-10-01,2017-07-09,2017-07-11,2017-07-14,2017-10-03,2017-11-01
1,2018,2018-03-29,2018-10-28,2018-02-21,2018-03-27,2018-03-29,2018-10-01,2018-07-15,2018-07-17,2018-07-19,2018-10-02,2018-10-28
2,2019,2019-03-20,2019-10-30,2019-02-21,2019-03-26,2019-03-20,2019-09-29,2019-07-07,2019-07-09,2019-07-11,2019-10-01,2019-10-30
3,2020,2020-07-23,2020-10-28,2020-02-21,2020-07-22,2020-07-23,2020-09-27,2020-08-25,NaN,2020-08-26,2020-09-29,2020-10-28
4,2021,2021-02-28,2021-10-31,2021-02-28,2021-03-30,2021-04-01,2021-10-03,2021-07-11,2021-07-13,2021-07-15,2021-10-04,2021-10-31


In [18]:
df_seasons.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seasonId                5 non-null      int64 
 1   seasonStartDate         5 non-null      object
 2   seasonEndDate           5 non-null      object
 3   preSeasonStartDate      5 non-null      object
 4   preSeasonEndDate        5 non-null      object
 5   regularSeasonStartDate  5 non-null      object
 6   regularSeasonEndDate    5 non-null      object
 7   lastDate1stHalf         5 non-null      object
 8   allStarDate             4 non-null      object
 9   firstDate2ndHalf        5 non-null      object
 10  postSeasonStartDate     5 non-null      object
 11  postSeasonEndDate       5 non-null      object
dtypes: int64(1), object(11)
memory usage: 608.0+ bytes


optamos por não utilizar o dataframe de Seasons, já que ele apenas adicionaria complexidade ao modelo e como se tratam de apenas datas, provavelmente a contribuição das features que ele adiciona não seriam relevantes já que são datas de início e fim de eventos pré definidos na temporada

In [19]:
del df_seasons

## Loading the Train dataset

In [20]:
# %%time
# df_train = pd.read_csv(dataset_names['Train'])

In [22]:
%%time
df_train = pd.read_pickle(raw_data_path + 'train.pkl')

CPU times: user 1.46 s, sys: 3.86 s, total: 5.32 s
Wall time: 9.26 s


### Dataset info

In [23]:
df_train.head()

,date,nextDayPlayerEngagement,games,rosters,playerBoxScores,teamBoxScores,transactions,standings,awards,events,playerTwitterFollowers,teamTwitterFollowers
0,20180101,"[{""engagementMetricsDate"":""2018-01-02"",""player...",NaN,"[{""playerId"":400121,""gameDate"":""2018-01-01"",""t...",NaN,NaN,"[{""transactionId"":340732,""playerId"":547348,""pl...",NaN,NaN,NaN,"[{""date"":""2018-01-01"",""playerId"":545361,""playe...","[{""date"":""2018-01-01"",""teamId"":147,""teamName"":..."
1,20180102,"[{""engagementMetricsDate"":""2018-01-03"",""player...",NaN,"[{""playerId"":134181,""gameDate"":""2018-01-02"",""t...",NaN,NaN,"[{""transactionId"":339458,""playerId"":621173,""pl...",NaN,NaN,NaN,NaN,NaN
2,20180103,"[{""engagementMetricsDate"":""2018-01-04"",""player...",NaN,"[{""playerId"":425492,""gameDate"":""2018-01-03"",""t...",NaN,NaN,"[{""transactionId"":347527,""playerId"":572389,""pl...",NaN,NaN,NaN,NaN,NaN
3,20180104,"[{""engagementMetricsDate"":""2018-01-05"",""player...",NaN,"[{""playerId"":282332,""gameDate"":""2018-01-04"",""t...",NaN,NaN,"[{""transactionId"":339549,""playerId"":545343,""pl...",NaN,NaN,NaN,NaN,NaN
4,20180105,"[{""engagementMetricsDate"":""2018-01-06"",""player...",NaN,"[{""playerId"":282332,""gameDate"":""2018-01-05"",""t...",NaN,NaN,"[{""transactionId"":341195,""playerId"":628336,""pl...",NaN,NaN,NaN,NaN,NaN


In [24]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1216 entries, 0 to 1215
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   date                     1216 non-null   int64 
 1   nextDayPlayerEngagement  1216 non-null   object
 2   games                    639 non-null    object
 3   rosters                  1216 non-null   object
 4   playerBoxScores          538 non-null    object
 5   teamBoxScores            538 non-null    object
 6   transactions             1103 non-null   object
 7   standings                531 non-null    object
 8   awards                   294 non-null    object
 9   events                   536 non-null    object
 10  playerTwitterFollowers   40 non-null     object
 11  teamTwitterFollowers     40 non-null     object
dtypes: int64(1), object(11)
memory usage: 114.1+ KB


In [25]:
df_train['nextDayPlayerEngagement'][0][:1000]

'[{"engagementMetricsDate":"2018-01-02","playerId":628317,"target1":0.011167070542384616,"target2":4.4747081712062258,"target3":0.0051677297424994094,"target4":5.7352941176470589},{"engagementMetricsDate":"2018-01-02","playerId":547989,"target1":0.042993221588180773,"target2":5.5933852140077827,"target3":0.045033073470351993,"target4":2.7941176470588238},{"engagementMetricsDate":"2018-01-02","playerId":519317,"target1":0.97432690482305784,"target2":56.177042801556418,"target3":13.693745570517363,"target4":64.166666666666671},{"engagementMetricsDate":"2018-01-02","playerId":607625,"target1":0.0067002423254307695,"target2":2.6750972762645913,"target3":0.0051677297424994094,"target4":1.8627450980392157},{"engagementMetricsDate":"2018-01-02","playerId":592547,"target1":0.0011167070542384616,"target2":0.632295719844358,"target3":0.0029529884242853769,"target4":0.93137254901960786},{"engagementMetricsDate":"2018-01-02","playerId":641553,"target1":0.011725424069503847,"target2":3.842412451361

### Auxiliary functions

In [26]:
def unpack_json(json_str):
    return pd.DataFrame() if pd.isna(json_str) else pd.read_json(json_str)

def unpack_data(data, dfs=None, n_jobs=-1):
    if dfs is not None:
        data = data.loc[:, dfs]
    unnested_dfs = {}
    for name, column in data.iteritems():
        daily_dfs = Parallel(n_jobs=n_jobs)(
            delayed(unpack_json)(item) for date, item in column.iteritems())
        df = pd.concat(daily_dfs)
        unnested_dfs[name] = df
    return unnested_dfs

### Making the targets df

In [27]:
%%time

Y = unpack_data(df_train, dfs = ['nextDayPlayerEngagement'])['nextDayPlayerEngagement']
Y = Y.astype({name: np.float32 for name in ["target1", "target2", "target3", "target4"]})
# Match target dates to feature dates and create date index
Y = Y.rename(columns={'engagementMetricsDate': 'date'})
Y['date'] = pd.to_datetime(Y['date'])
Y = Y.set_index('date').to_period('D')
Y.index = Y.index - 1
Y = Y.reset_index()


CPU times: user 4.04 s, sys: 1.83 s, total: 5.87 s
Wall time: 10.6 s


In [28]:
Y.head()

,date,playerId,target1,target2,target3,target4
0,2018-01-01,628317,0.011167,4.474708,0.005168,5.735294
1,2018-01-01,547989,0.042993,5.593385,0.045033,2.794118
2,2018-01-01,519317,0.974327,56.177044,13.693746,64.166664
3,2018-01-01,607625,0.006700,2.675097,0.005168,1.862745
4,2018-01-01,592547,0.001117,0.632296,0.002953,0.931373


In [29]:
Y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2506176 entries, 0 to 2506175
Data columns (total 6 columns):
 #   Column    Dtype    
---  ------    -----    
 0   date      period[D]
 1   playerId  int64    
 2   target1   float32  
 3   target2   float32  
 4   target3   float32  
 5   target4   float32  
dtypes: float32(4), int64(1), period[D](1)
memory usage: 76.5 MB


In [37]:
# pd.to_pickle(Y, processed_data_path + 'targets.pkl')
del Y

### Player Box Scores

In [50]:
%%time
df_playerBoxScores = unpack_data(df_train, dfs = ['playerBoxScores'])['playerBoxScores']

CPU times: user 2.21 s, sys: 1.25 s, total: 3.46 s
Wall time: 9.23 s


In [51]:
df_playerBoxScores.head()

,home,gamePk,gameDate,gameTimeUTC,teamId,teamName,playerId,playerName,jerseyNum,positionCode,...,catchersInterferencePitching,sacBuntsPitching,sacFliesPitching,saves,holds,blownSaves,assists,putOuts,errors,chances
0,1,529418,2018-03-29,2018-03-29T23:08:00Z,119,Los Angeles Dodgers,605131,Austin Barnes,15,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,529406,2018-03-29,2018-03-29T20:00:00Z,139,Tampa Bay Rays,605480,Mallex Smith,0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
2,0,529416,2018-03-29,2018-03-29T20:10:00Z,143,Philadelphia Phillies,546318,Odubel Herrera,37,8,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
3,0,529412,2018-03-29,2018-03-29T20:05:00Z,108,Los Angeles Angels,527043,Jefry Marte,19,3,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0
4,1,529408,2018-03-29,2018-03-29T20:15:00Z,118,Kansas City Royals,449181,Paulo Orlando,16,8,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.0,0.0,2.0


In [52]:
df_playerBoxScores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185144 entries, 0 to 430
Data columns (total 85 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   home                          185144 non-null  int64  
 1   gamePk                        185144 non-null  int64  
 2   gameDate                      185144 non-null  object 
 3   gameTimeUTC                   185144 non-null  object 
 4   teamId                        185144 non-null  int64  
 5   teamName                      185144 non-null  object 
 6   playerId                      185144 non-null  int64  
 7   playerName                    185144 non-null  object 
 8   jerseyNum                     185112 non-null  object 
 9   positionCode                  185144 non-null  int64  
 10  positionName                  185144 non-null  object 
 11  positionType                  185144 non-null  object 
 12  battingOrder                  153749 non-null  

In [53]:
cols = {
    # columns related to other dimensions
    'gamePk': 'IdGame',
    'gameDate': 'DtGame',
    'gameTimeUTC': 'DtGameUTC',
    'playerId': 'IdPlayer',
    'teamId': 'IdTeam',
    'jerseyNum': 'NuJersey',
    'positionCode': 'CdPosition',
}  
# numeric columns
for numeric_col in list(df_playerBoxScores.columns[12:]):
    cols[numeric_col] = 'Nu' + numeric_col[0].upper() + numeric_col[1:]
print(cols)

{'gamePk': 'IdGame', 'gameDate': 'DtGame', 'gameTimeUTC': 'DtGameUTC', 'playerId': 'IdPlayer', 'teamId': 'IdTeam', 'jerseyNum': 'NuJersey', 'positionCode': 'CdPosition', 'battingOrder': 'NuBattingOrder', 'gamesPlayedBatting': 'NuGamesPlayedBatting', 'flyOuts': 'NuFlyOuts', 'groundOuts': 'NuGroundOuts', 'runsScored': 'NuRunsScored', 'doubles': 'NuDoubles', 'triples': 'NuTriples', 'homeRuns': 'NuHomeRuns', 'strikeOuts': 'NuStrikeOuts', 'baseOnBalls': 'NuBaseOnBalls', 'intentionalWalks': 'NuIntentionalWalks', 'hits': 'NuHits', 'hitByPitch': 'NuHitByPitch', 'atBats': 'NuAtBats', 'caughtStealing': 'NuCaughtStealing', 'stolenBases': 'NuStolenBases', 'groundIntoDoublePlay': 'NuGroundIntoDoublePlay', 'groundIntoTriplePlay': 'NuGroundIntoTriplePlay', 'plateAppearances': 'NuPlateAppearances', 'totalBases': 'NuTotalBases', 'rbi': 'NuRbi', 'leftOnBase': 'NuLeftOnBase', 'sacBunts': 'NuSacBunts', 'sacFlies': 'NuSacFlies', 'catchersInterference': 'NuCatchersInterference', 'pickoffs': 'NuPickoffs', 'g

In [54]:
df_playerBoxScores = df_playerBoxScores[list(cols)]
df_playerBoxScores.columns = list(cols.values())

In [55]:
# pd.to_pickle(df_playerBoxScores, processed_data_path + 'playerBoxScores.pkl')
del df_playerBoxScores